In [7]:
#Recoleccion de datos 
import requests
import pandas as pd
import time

years = range(1980, 2025)  
months = range(1, 13)  
days = range(1, 32) 
pause_time = 0.5 

base_games_url = "https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard"
teams_url = "https://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams"

games_list = []
teams_list = []
players_list = []
player_stats_list = []

print("Obteniendo datos de equipos...")
teams_response = requests.get(teams_url)

if teams_response.status_code == 200:
    teams_data = teams_response.json().get("sports", [])[0].get("leagues", [])[0].get("teams", [])
    
    for team in teams_data:
        team_info = team.get("team", {})
        team_entry = {
            "team_id": team_info.get("id"),
            "team_name": team_info.get("displayName"),
            "team_abbreviation": team_info.get("abbreviation"),
            "location": team_info.get("location"),
            "nickname": team_info.get("nickname"),
            "record": team_info.get("record", {}).get("summary", "N/A")
        }
        teams_list.append(team_entry)
        
        team_roster_url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams/{team_info.get('id')}/roster"
        roster_response = requests.get(team_roster_url)
        
        if roster_response.status_code == 200:
            roster_data = roster_response.json().get("athletes", [])
            
            for player in roster_data:
                player_info = player.get("athlete", {})
                player_entry = {
                    "player_id": player_info.get("id"),
                    "full_name": player_info.get("displayName"),
                    "position": player_info.get("position", {}).get("abbreviation", "N/A"),
                    "team_id": team_info.get("id"),
                    "height": player_info.get("height", "N/A"),
                    "weight": player_info.get("weight", "N/A"),
                    "age": player_info.get("age", "N/A")
                }
                players_list.append(player_entry)

        time.sleep(pause_time)

print(f"Se obtuvieron {len(teams_list)} equipos y {len(players_list)} jugadores.")

print("Obteniendo datos de juegos...")
for year in years:
    for month in months:
        for day in days:
            date = f"{year}{month:02d}{day:02d}"
            url = f"{base_games_url}?dates={date}"
            
            try:
                response = requests.get(url)
                if response.status_code == 200:
                    data = response.json()
                    games = data.get("events", [])

                    for game in games:
                        game_id = game["id"]
                        game_info = {
                            "game_id": game_id,
                            "date": game["date"],
                            "status": game["status"]["type"]["description"],
                            "home_team": game["competitions"][0]["competitors"][0]["team"]["displayName"],
                            "home_score": game["competitions"][0]["competitors"][0]["score"],
                            "away_team": game["competitions"][0]["competitors"][1]["team"]["displayName"],
                            "away_score": game["competitions"][0]["competitors"][1]["score"]
                        }
                        games_list.append(game_info)

                        game_stats_url = f"https://site.api.espn.com/apis/site/v2/sports/basketball/nba/summary?event={game_id}"
                        stats_response = requests.get(game_stats_url)

                        if stats_response.status_code == 200:
                            stats_data = stats_response.json().get("boxscore", {}).get("players", [])

                            for team in stats_data:
                                for player in team.get("statistics", []):
                                    player_info = player.get("athlete", {})
                                    player_stats = {
                                        "game_id": game_id,
                                        "player_id": player_info.get("id"),
                                        "full_name": player_info.get("displayName"),
                                        "team": player_info.get("team", {}).get("displayName"),
                                        "points": player.get("stats", {}).get("points", 0),
                                        "rebounds": player.get("stats", {}).get("rebounds", 0),
                                        "assists": player.get("stats", {}).get("assists", 0),
                                        "blocks": player.get("stats", {}).get("blocks", 0),
                                        "steals": player.get("stats", {}).get("steals", 0),
                                        "minutes": player.get("stats", {}).get("minutes", "0:00")
                                    }
                                    player_stats_list.append(player_stats)

                print(f"Datos obtenidos para {date}: {len(games_list)} juegos en total")
                time.sleep(pause_time)

            except Exception as e:
                print(f"Error en {date}: {e}")

print(f"Se obtuvieron {len(games_list)} juegos y {len(player_stats_list)} estadísticas de jugadores.")

pd.DataFrame(teams_list).to_csv("team_history.csv", index=False)
pd.DataFrame(players_list).to_csv("player.csv", index=False)
pd.DataFrame(games_list).to_csv("draft_history.csv", index=False)
pd.DataFrame(player_stats_list).to_csv("common_player_info.csv", index=False)

print("Archivos guardados: nba_teams.csv, nba_players.csv, nba_games.csv, nba_player_stats.csv")


📌 Obteniendo datos de equipos...
✅ Se obtuvieron 30 equipos y 528 jugadores.
📌 Obteniendo datos de juegos...
📅 Datos obtenidos para 19800101: 0 juegos en total
📅 Datos obtenidos para 19800102: 2 juegos en total
📅 Datos obtenidos para 19800103: 2 juegos en total
📅 Datos obtenidos para 19800104: 2 juegos en total
📅 Datos obtenidos para 19800105: 3 juegos en total
📅 Datos obtenidos para 19800106: 4 juegos en total
📅 Datos obtenidos para 19800107: 4 juegos en total
📅 Datos obtenidos para 19800108: 4 juegos en total
📅 Datos obtenidos para 19800109: 6 juegos en total
📅 Datos obtenidos para 19800110: 6 juegos en total
📅 Datos obtenidos para 19800111: 8 juegos en total
📅 Datos obtenidos para 19800112: 8 juegos en total
📅 Datos obtenidos para 19800113: 9 juegos en total
📅 Datos obtenidos para 19800114: 9 juegos en total
📅 Datos obtenidos para 19800115: 10 juegos en total
📅 Datos obtenidos para 19800116: 12 juegos en total
📅 Datos obtenidos para 19800117: 12 juegos en total
📅 Datos obtenidos par

In [2]:
#Concatenacion de datos
import pandas as pd
import json

common_player_info = pd.read_csv('common_player_info.csv')
draft_history = pd.read_csv('draft_history.csv')
player = pd.read_csv('player.csv')
team = pd.read_csv('team.csv')
team_history = pd.read_csv('team_history.csv')

combined_df = pd.concat([common_player_info, draft_history, player, team, team_history], ignore_index=True)

combined_json = combined_df.to_json(orient='records', force_ascii=False)

with open('combined_data.json', 'w', encoding='utf-8') as f:
    f.write(combined_json)

print("Los archivos CSV han sido combinados y guardados en 'combined_data.json'")


Los archivos CSV han sido combinados y guardados en 'combined_data.json'


In [3]:
#Limpieza de Datos 

import pandas as pd

# Cargar el archivo JSON en un DataFrame
with open('combined_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Eliminar filas con valores nulos
df_cleaned = df.dropna()

# Eliminar filas duplicadas
df_cleaned = df_cleaned.drop_duplicates()

# Guardar el DataFrame limpio en un nuevo archivo JSON
df_cleaned.to_json('cleaned_data.json', orient='records', force_ascii=False)

print("Los datos han sido limpiados y guardados en 'cleaned_data.json'")

Los datos han sido limpiados y guardados en 'cleaned_data.json'
